In [4]:
import pygame, random
from pygame.locals import *

SCREEN_WIDTH = 1200
SCREEN_HEIGHT = 700

fundo = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

moedas_p = 0
vidas_p  = 0

class Player(pygame.sprite.Sprite):

    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.images = [pygame.image.load('Aazul.png').convert_alpha()]
        self.current_image = 0
        self.image = pygame.image.load('Aazul.png').convert_alpha()
        self.rect = self.image.get_rect()
        self.rect[0] = 10
        self.rect[1] = 100
        self.fall = False
        self.vidas_p = 1
        
    def update(self):
        self.image = self.images[self.current_image]

        # Update height
        if (pygame.sprite.groupcollide(player_group, 
                                       ground_group, 
                                       False, 
                                       False, 
                                       pygame.sprite.collide_mask) or
            pygame.sprite.groupcollide(player_group, 
                                       box_group, 
                                       False, 
                                       False, 
                                       pygame.sprite.collide_mask) or
            isjump):  
            return
        
        # gravidade ou não
        if isjump == False: 
            self.rect[1] += 5

# Chão:                
w0 = []
for i in range(5):w0.append([i*150, 550])
ground = []
for i in range(len(w0)):
    class Ground(pygame.sprite.Sprite):
        def __init__(self):
            pygame.sprite.Sprite.__init__(self)
            self.images = [pygame.image.load('terreno.png').convert_alpha()]
            self.current_image = 0
            self.image = pygame.image.load('terreno.png').convert_alpha()
            self.mask = pygame.mask.from_surface(self.image)
            self.rect = self.image.get_rect()
            self.rect[0] = w0[i][0]
            self.rect[1] = w0[i][1]
        
        def update(self): self.image = self.images[self.current_image]
            
        def go_on(self):
            if (pygame.sprite.groupcollide(box_group, 
                                           player_group, 
                                           False, 
                                           False, 
                                           pygame.sprite.collide_mask)): 
                return
            self.rect[0] -= 5
                
        def go_back(self):
            if (pygame.sprite.groupcollide(box_group, 
                                           player_group, 
                                           False, 
                                           False, 
                                           pygame.sprite.collide_mask)): 
                return
            self.rect[0] += 5
            
    ground.append(Ground())
        
# Objeto inanimado:              
w1 =[] 
box = []
                
for i in range(len(w1)):
    class Box(pygame.sprite.Sprite):
        def __init__(self):
            pygame.sprite.Sprite.__init__(self)
            self.images = [pygame.image.load('Amarrom.png').convert_alpha()]
            self.current_image = 0
            self.image = pygame.image.load('Amarrom.png').convert_alpha()
            self.mask = pygame.mask.from_surface(self.image)
            self.rect = self.image.get_rect()
            self.rect[0] = w1[i][0]
            self.rect[1] = w1[i][1]

        def update(self): self.image = self.images[self.current_image]
            
    box.append(Box())

# Preparo das aparências do jogo:  
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, 
                                  SCREEN_HEIGHT))

# Jogador
player_group = pygame.sprite.Group()
player = Player()
player_group.add(player)

# Terreno
ground_group = pygame.sprite.Group()
ground_group.add(ground)

# Objeto Inanimado
box_group = pygame.sprite.Group()
box_group.add(box)

clock = pygame.time.Clock()    
isjump = False

def jogo(): 
    
    # Marcador de tempo:
    timer = 0
    tempo_segundos = 0

    # variáveis da desgraça do pulo:
    jumpcount = 20
    v = []    
    while jumpcount >= -20:    
        if jumpcount < 0: neg = -1
        else: neg = 1
        v.append((jumpcount**2)*0.5*neg/8)
        jumpcount -= 1
        
    jumpcount = 20
    i = 0
    j = 0
    s = []
    isjump = False
    ###############################
    
    # parte original    
    fimdejogo = False
    play = True
    while play:
        
        # regulador de velocidade de jogo: 
        clock.tick(50)
        #################################
        
        # caso se feche a janela:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
        #################################
        
        # game over!!
        if player.vidas_p == 0: fimdejogo = True
        if fimdejogo == True:
                fundo.fill((255,255,255))
                texto("Fim de jogo, para continuar tecle RIGHT ou LEFT para sair", (0,0,0))
                pygame.display.update()  
            
                comandos = pygame.key.get_pressed()
                
                if comandos[pygame.K_RIGHT]:
                    player.vidas_p += 1
                    player.rect[0] = 10
                    player.rect[1] = GROUND_HEIGHT - 105
                    jogo()
                            
                if comandos[pygame.K_LEFT]:
                            play = False
                            fimdejogo = False
        ###############
        
        #se ainda houver vidas ou continue:
        else:
    
            # comandos do jogador:    
            comandos = pygame.key.get_pressed()   
            
            if comandos[pygame.K_LEFT]:
                for k in range(len(w0)):ground[k].go_back()
                for k in range(len(w1)):box[k].go_back()
            
            if comandos[pygame.K_RIGHT]:
                for k in range(len(w0)):ground[k].go_on()
                for k in range(len(w1)):box[k].go_on()
    
            # pulo
            if (isjump == False):
                if ((comandos[pygame.K_SPACE] and 
                    (pygame.sprite.groupcollide(player_group, 
                                                ground_group, 
                                                False, 
                                                False, 
                                                pygame.sprite.collide_mask) or 
                     pygame.sprite.groupcollide(player_group, 
                                                box_group, 
                                                False, 
                                                False, 
                                                pygame.sprite.collide_mask)))): 
                     isjump = True
            else:                      
                    s.append(v)
                    if jumpcount >= -20: 
                        if jumpcount < 0: neg = -1
                        else: neg = 1    
                        player.rect[1] -= s[j][i]
                
                        if pygame.sprite.groupcollide(player_group, 
                                                      box_group, 
                                                      False, 
                                                      False, 
                                                      pygame.sprite.collide_mask):
                            player.rect[1] += s[0][i]
                            
                        if pygame.sprite.groupcollide(player_group, 
                                                      ground_group, 
                                                      False, 
                                                      False, 
                                                      pygame.sprite.collide_mask):
                            player.rect[1] += s[0][i]
                    
                        jumpcount -= 1
                        i += 1
                    else: 
                        i = 0
                        s.remove(v)
                        jumpcount = 20
                        isjump = False
            # Fim do pulo
    
            # imagens e atualização
            fundo.fill((127,127,127))
  
            player_group.update()
            box_group.update()
            ground_group.update()

            ground_group.draw(fundo)
            box_group.draw(fundo)
            player_group.draw(fundo)

            pygame.display.update()

jogo()
pygame.quit()

error: video system not initialized